In [10]:
import ast
import re
import io
import json
import gzip
import numpy as np
import pandas as pd
from google.cloud import storage

In [11]:
pd.set_option('max_colwidth', 999999)
pd.options.mode.copy_on_write = True

In [12]:
bucket_name = "chronos-cron-prod"
source = "combined-20240224.log.gz" # gs uri

# Explicitly use service account credentials by specifying the private key file.
storage_client = storage.Client.from_service_account_json('/content/perqara-data-532572ce4996.json')

# Get the bucket and blob objects
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(source)

# Download the contents of the blob as a string
str_data = io.BytesIO(blob.download_as_string())

In [13]:
# Open gzip into csv
with gzip.open(str_data) as gz:
  # Read compressed file as a file object
  file = gz.read()
  # Decode the byte type into string by utf-8
  blob_decompress = file.decode('utf-8')
  # StringIO object
  s = io.StringIO(blob_decompress)

In [14]:
df = pd.read_json(s, dtype={'codename': str, 'environment': str, 'level': str, 'message': str, 'meta': str}, nrows=999999999, lines=True)

In [15]:
df

,codename,environment,level,message,meta
0,chronos,production,info,HTTP GET /,"{'req': {'headers': {'accept': '*/*', 'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': 'chronos.perqara.com', 'user-agent': 'Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 'x-real-ip': '23.88.41.31'}, 'httpVersion': '1.1', 'method': 'GET', 'originalUrl': '/', 'query': {}, 'url': '/'}, 'res': {'statusCode': 200}, 'responseTime': 1}"
1,chronos,production,info,HTTP GET /,"{'req': {'headers': {'accept': '*/*', 'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': 'chronos.perqara.com', 'user-agent': 'Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 'x-real-ip': '143.42.97.120'}, 'httpVersion': '1.1', 'method': 'GET', 'originalUrl': '/', 'query': {}, 'url': '/'}, 'res': {'statusCode': 200}, 'responseTime': 1}"
2,chronos,production,info,HTTP HEAD /,"{'req': {'headers': {'accept': '*/*', 'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': 'chronos.perqara.com', 'user-agent': 'Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 'x-real-ip': '172.104.109.161'}, 'httpVersion': '1.1', 'method': 'HEAD', 'originalUrl': '/', 'query': {}, 'url': '/'}, 'res': {'statusCode': 200}, 'responseTime': 0}"
3,chronos,production,info,HTTP GET /,"{'req': {'headers': {'accept': '*/*', 'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': 'chronos.perqara.com', 'user-agent': 'Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 'x-real-ip': '172.105.169.250'}, 'httpVersion': '1.1', 'method': 'GET', 'originalUrl': '/', 'query': {}, 'url': '/'}, 'res': {'statusCode': 200}, 'responseTime': 1}"
4,chronos,production,info,HTTP GET /,"{'req': {'headers': {'accept': '*/*', 'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': 'chronos.perqara.com', 'user-agent': 'Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 'x-real-ip': '172.104.50.111'}, 'httpVersion': '1.1', 'method': 'GET', 'originalUrl': '/', 'query': {}, 'url': '/'}, 'res': {'statusCode': 200}, 'responseTime': 0}"
...,...,...,...,...,...
729,chronos,production,info,HTTP GET /LDRsJN,"{'req': {'headers': {'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': '34.101.39.29', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363', 'x-real-ip': '184.95.51.10'}, 'httpVersion': '1.1', 'method': 'GET', 'originalUrl': '/LDRsJN', 'query': {}, 'url': '/LDRsJN'}, 'res': {'statusCode': 404}, 'responseTime': 3}"
730,chronos,production,info,HTTP GET /heapdump,"{'req': {'headers': {'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': '34.101.39.29', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) AppleWebKit/537.36 (KHTML, like Gecko) Version/13.0.63 Safari/632.12', 'x-real-ip': '184.95.51.10'}, 'httpVersion': '1.1', 'method': 'GET', 'originalUrl': '/heapdump', 'query': {}, 'url': '/heapdump'}, 'res': {'statusCode': 404}, 'responseTime': 0}"
731,chronos,production,info,HTTP GET /actuator/heapdump,"{'req': {'headers': {'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': '34.101.39.29', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0 ADFS-SSO-DS', 'x-real-ip': '184.95.51.10'}, 'httpVersion': '1.1', 'method': 'GET', 'originalUrl': '/actuator/heapdump', 'query': {}, 'url': '/actuator/heapdump'}, 'res': {'statusCode': 404}, 'responseTime': 0}"
732,chronos,production,info,HTTP GET /,"{'req': {'headers': {'accept': '*/*', 'accept-encoding': 'gzip', 'connection': 'upgrade', 'ho

In [21]:
# Assuming df is your DataFrame and 'json_column' is the column containing JSON data
normalized_df = pd.json_normalize(df['meta'].apply(ast.literal_eval), max_level=5)

In [22]:
normalized_df

,responseTime,req.headers.accept,req.headers.accept-encoding,req.headers.connection,req.headers.host,req.headers.user-agent,req.headers.x-real-ip,req.httpVersion,req.method,req.originalUrl,...,req.query.form,req.query.operation,req.query.lang,req.query.panel,req.query. config-create /,"req.query./<?echo(md5(""hi""));?> /var/tmp/index1.php",req.query.function,req.query.s,req.query.vars,req.headers.authorization
0,1,*/*,gzip,upgrade,chronos.perqara.com,"Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",23.88.41.31,1.1,GET,/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,*/*,gzip,upgrade,chronos.perqara.com,"Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",143.42.97.120,1.1,GET,/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,*/*,gzip,upgrade,chronos.perqara.com,"Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",172.104.109.161,1.1,HEAD,/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,*/*,gzip,upgrade,chronos.perqara.com,"Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",172.105.169.250,1.1,GET,/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,*/*,gzip,upgrade,chronos.perqara.com,"Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",172.104.50.111,1.1,GET,/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,3,NaN,gzip,upgrade,34.101.39.29,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363",184.95.51.10,1.1,GET,/LDRsJN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
730,0,NaN,gzip,upgrade,34.101.39.29,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) AppleWebKit/537.36 (KHTML, like Gecko) Version/13.0.63 Safari/632.12",184.95.51.10,1.1,GET,/heapdump,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
731,0,NaN,gzip,upgrade,34.101.39.29,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0 ADFS-SSO-DS",184.95.51.10,1.1,GET,/actuator/heapdump,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
732,0,*/*,gzip,upgrade,chronos.perqara.com,"Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",139.162.109.252,1.1,GET,/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
normalized_df.columns

Index(['responseTime', 'req.headers.accept', 'req.headers.accept-encoding',
       'req.headers.connection', 'req.headers.host', 'req.headers.user-agent',
       'req.headers.x-real-ip', 'req.httpVersion', 'req.method',
       'req.originalUrl', 'req.url', 'res.statusCode', 'req.headers.referer',
       'req.headers.x-datadog-parent-id',
       'req.headers.x-datadog-sampling-priority',
       'req.headers.x-datadog-trace-id', 'req.headers.content-length',
       'req.headers.content-type', 'req.headers.accept-charset',
       'req.headers.accept-language', 'req.headers.cache-control',
       'req.headers.pragma', 'req.headers.upgrade-insecure-requests',
       'req.query.2clbbZYsKKmA9EmRajnV9kMk4L4', 'req.headers.x-ssl-vpn',
       'req.query.country', 'req.query.form', 'req.query.operation',
       'req.query.lang', 'req.query.panel', 'req.query. config-create /',
       'req.query./<?echo(md5("hi"));?> /var/tmp/index1.php',
       'req.query.function', 'req.query.s', 'req.query.vars

In [24]:
df['user_agent'] = normalized_df['req.headers.user-agent'].to_numpy()

In [25]:
df

,codename,environment,level,message,meta,user_agent
0,chronos,production,info,HTTP GET /,"{'req': {'headers': {'accept': '*/*', 'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': 'chronos.perqara.com', 'user-agent': 'Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 'x-real-ip': '23.88.41.31'}, 'httpVersion': '1.1', 'method': 'GET', 'originalUrl': '/', 'query': {}, 'url': '/'}, 'res': {'statusCode': 200}, 'responseTime': 1}","Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
1,chronos,production,info,HTTP GET /,"{'req': {'headers': {'accept': '*/*', 'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': 'chronos.perqara.com', 'user-agent': 'Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 'x-real-ip': '143.42.97.120'}, 'httpVersion': '1.1', 'method': 'GET', 'originalUrl': '/', 'query': {}, 'url': '/'}, 'res': {'statusCode': 200}, 'responseTime': 1}","Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
2,chronos,production,info,HTTP HEAD /,"{'req': {'headers': {'accept': '*/*', 'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': 'chronos.perqara.com', 'user-agent': 'Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 'x-real-ip': '172.104.109.161'}, 'httpVersion': '1.1', 'method': 'HEAD', 'originalUrl': '/', 'query': {}, 'url': '/'}, 'res': {'statusCode': 200}, 'responseTime': 0}","Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
3,chronos,production,info,HTTP GET /,"{'req': {'headers': {'accept': '*/*', 'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': 'chronos.perqara.com', 'user-agent': 'Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 'x-real-ip': '172.105.169.250'}, 'httpVersion': '1.1', 'method': 'GET', 'originalUrl': '/', 'query': {}, 'url': '/'}, 'res': {'statusCode': 200}, 'responseTime': 1}","Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
4,chronos,production,info,HTTP GET /,"{'req': {'headers': {'accept': '*/*', 'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': 'chronos.perqara.com', 'user-agent': 'Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', 'x-real-ip': '172.104.50.111'}, 'httpVersion': '1.1', 'method': 'GET', 'originalUrl': '/', 'query': {}, 'url': '/'}, 'res': {'statusCode': 200}, 'responseTime': 0}","Better Uptime Bot Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
...,...,...,...,...,...,...
729,chronos,production,info,HTTP GET /LDRsJN,"{'req': {'headers': {'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': '34.101.39.29', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363', 'x-real-ip': '184.95.51.10'}, 'httpVersion': '1.1', 'method': 'GET', 'originalUrl': '/LDRsJN', 'query': {}, 'url': '/LDRsJN'}, 'res': {'statusCode': 404}, 'responseTime': 3}","Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363"
730,chronos,production,info,HTTP GET /heapdump,"{'req': {'headers': {'accept-encoding': 'gzip', 'connection': 'upgrade', 'host': '34.101.39.29', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) AppleWebKit/537.36 (KHTML, like Gecko) Version/13.0.63 Safari/